A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [1]:
from mingpt.utils import set_seed
from mingpt.model import GPT
from mingpt.trainer import Trainer

In [2]:
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader


import pickle
import random

set_seed(3407)

In [3]:
# class SortDataset(Dataset):
#     """ 
#     Dataset for the Sort problem. E.g. for problem length 6:
#     Input: 0 0 2 1 0 1 -> Output: 0 0 0 1 1 2
#     Which will feed into the transformer concatenated as:
#     input:  0 0 2 1 0 1 0 0 0 1 1
#     output: I I I I I 0 0 0 1 1 2
#     where I is "ignore", as the transformer is reading the input sequence
#     """

#     def __init__(self, split, length=6, num_digits=3):
#         assert split in {'train', 'test'}
#         self.split = split
#         self.length = length
#         self.num_digits = num_digits
    
#     def __len__(self):
#         return 10000 # ...
    
#     def get_vocab_size(self):
#         return self.num_digits
    
#     def get_block_size(self):
#         # the length of the sequence that will feed into transformer, 
#         # containing concatenated input and the output, but -1 because
#         # the transformer starts making predictions at the last input element
#         return self.length * 2 - 1

#     def __getitem__(self, idx):
        
#         # use rejection sampling to generate an input example from the desired split
#         while True:
#             # generate some random integers
#             inp = torch.randint(self.num_digits, size=(self.length,), dtype=torch.long)
#             # half of the time let's try to boost the number of examples that 
#             # have a large number of repeats, as this is what the model seems to struggle
#             # with later in training, and they are kind of rate
#             if torch.rand(1).item() < 0.5:
#                 if inp.unique().nelement() > self.length // 2:
#                     # too many unqiue digits, re-sample
#                     continue
#             # figure out if this generated example is train or test based on its hash
#             h = hash(pickle.dumps(inp.tolist()))
#             inp_split = 'test' if h % 4 == 0 else 'train' # designate 25% of examples as test
#             if inp_split == self.split:
#                 break # ok
        
#         # solve the task: i.e. sort
#         sol = torch.sort(inp)[0]

#         # concatenate the problem specification and the solution
#         cat = torch.cat((inp, sol), dim=0)

#         # the inputs to the transformer will be the offset sequence
#         x = cat[:-1].clone()
#         y = cat[1:].clone()
#         # we only want to predict at output locations, mask out the loss at the input locations
#         y[:self.length-1] = -1
#         return x, y


In [3]:
def pad_with_zeros_str_mult(s, length):
    return (2 * length - len(s)) * '0' + s if (2 * length - len(s)) >0 else s

def random_add_instance(length):
    a = [random.randint(0,9) for i in range(length)]
    b = [random.randint(0,9) for i in range(length)]
    val_a = int(''.join(str(d) for d in a))
    val_b = int(''.join(str(d) for d in b))
    val_c = val_a + val_b
    str_c = str(val_c)
    str_c = (length + 1 - len(str_c)) * '0' + str_c
    return a + b + [int(d) for d in str_c]

def random_multiply_instance(length):
    a = [random.randint(0,9) for i in range(length)]
    b = [random.randint(0,9) for i in range(length)]
    val_a = int(''.join(str(d) for d in a))
    val_b = int(''.join(str(d) for d in b))
    val_c = val_a * val_b
    str_c = str(val_c)
    str_c = (2 * length - len(str_c)) * '0' + str_c if (2 * length - len(str_c)) >0 else str_c
    return a + b + [int(d) for d in str_c]

def random_multiply_long_instance(length):
    a = [random.randint(0,9) for i in range(length)]
    b = [random.randint(0,9) for i in range(length)]
    # a = [1, 2 , 3]
    # b = [4, 5, 6]
    val_a = int(''.join(str(d) for d in a))
    val_b = int(''.join(str(d) for d in b))
    elems = []
    ten_pow = 1
    for d in b[::-1]:
        e_val = val_a * d * ten_pow
        e_str = pad_with_zeros_str_mult(str(e_val), length)
        elems.append(e_str)
        ten_pow *= 10

    val_c = val_a * val_b
    str_c = str(val_c)
    str_c = (2 * length - len(str_c)) * '0' + str_c if (2 * length - len(str_c)) >0 else str_c
    return a + b + [int(d) for d in "".join(elems)] + [int(d) for d in str_c]


for i in range(10):
#     # print (random_add_instance(3))
    print(random_multiply_long_instance(3))
#     print(random_multiply_instance(3))



[0, 6, 2, 5, 3, 6, 0, 0, 0, 3, 7, 2, 0, 0, 1, 8, 6, 0, 0, 3, 1, 0, 0, 0, 0, 3, 3, 2, 3, 2]
[4, 9, 5, 3, 7, 8, 0, 0, 3, 9, 6, 0, 0, 3, 4, 6, 5, 0, 1, 4, 8, 5, 0, 0, 1, 8, 7, 1, 1, 0]
[6, 4, 4, 3, 1, 4, 0, 0, 2, 5, 7, 6, 0, 0, 6, 4, 4, 0, 1, 9, 3, 2, 0, 0, 2, 0, 2, 2, 1, 6]
[3, 5, 9, 1, 1, 9, 0, 0, 3, 2, 3, 1, 0, 0, 3, 5, 9, 0, 0, 3, 5, 9, 0, 0, 0, 4, 2, 7, 2, 1]
[3, 5, 0, 1, 4, 8, 0, 0, 2, 8, 0, 0, 0, 1, 4, 0, 0, 0, 0, 3, 5, 0, 0, 0, 0, 5, 1, 8, 0, 0]
[0, 0, 2, 9, 3, 7, 0, 0, 0, 0, 1, 4, 0, 0, 0, 0, 6, 0, 0, 0, 1, 8, 0, 0, 0, 0, 1, 8, 7, 4]
[8, 4, 8, 2, 3, 7, 0, 0, 5, 9, 3, 6, 0, 2, 5, 4, 4, 0, 1, 6, 9, 6, 0, 0, 2, 0, 0, 9, 7, 6]
[9, 9, 9, 7, 0, 6, 0, 0, 5, 9, 9, 4, 0, 0, 0, 0, 0, 0, 6, 9, 9, 3, 0, 0, 7, 0, 5, 2, 9, 4]
[7, 4, 5, 1, 3, 6, 0, 0, 4, 4, 7, 0, 0, 2, 2, 3, 5, 0, 0, 7, 4, 5, 0, 0, 1, 0, 1, 3, 2, 0]
[1, 0, 1, 0, 4, 2, 0, 0, 0, 2, 0, 2, 0, 0, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 2, 4, 2]


In [5]:
# class AddDataset(Dataset):
#     """ 
#     Dataset for the Add problem. E.g. for problem length 3:
#     12 + 333 = 345
#     Input: 0 1 2 3 3 3 -> Output: 0 3 4 5
#     Which will feed into the transformer concatenated as:
#     input:  0 1 2 3 3 3 0 3 4
#     output: I I I I I 0 3 4 5
#     where I is "ignore", as the transformer is reading the input sequence
#     """

#     def __init__(self, split, length=3):
#         assert split in {'train', 'test'}
#         self.split = split
#         self.length = length
    
#     def __len__(self):
#         return 10000 # ...
    
#     def get_vocab_size(self):
#         return 10
    
#     def get_block_size(self):
#         # the length of the sequence that will feed into transformer, 
#         # containing concatenated input and the output, but -1 because
#         # the transformer starts making predictions at the last input element
#         return 3 * self.length + 1 - 1

#     def __getitem__(self, idx):
#         while True:
#             rai = random_add_instance(self.length)
#             h = hash(str(rai[:2*self.length]))
            
#             inp_split = 'test' if h % 4 == 0 else 'train' # designate 25% of examples as test
#             if inp_split == self.split:
#                 break # ok
        
#         x = torch.tensor(rai[:-1], dtype=torch.long)
#         y = torch.tensor(rai[1:], dtype=torch.long)
        
#         # we only want to predict at output locations, mask out the loss at the input locations
#         y[:2*self.length-1] = -1
#         return x, y

In [4]:
class MultiplyDataset(Dataset):
    """ 
    Dataset for the Multiply problem. E.g. for problem length 3:
    12 * 333 = 345
    Input: 0 1 2 3 3 3 -> Output: 0 0 3 9 9 6
    Which will feed into the transformer concatenated as:
    input:  0 1 2 3 3 3 0 0 3 9 9
    output: I I I I I 0 0 3 9 9 6
    where I is "ignore", as the transformer is reading the input sequence
    """

    def __init__(self, split, length=3):
        assert split in {'train', 'test'}
        self.split = split
        self.length = length
    
    def __len__(self):
        return 1000000 # ...
    
    def get_vocab_size(self):
        return 10
    
    def get_block_size(self):
        # the length of the sequence that will feed into transformer, 
        # containing concatenated input and the output, but -1 because
        # the transformer starts making predictions at the last input element
        return 4 * self.length - 1

    def __getitem__(self, idx):
        while True:
            rai = random_multiply_instance(self.length)
            h = hash(str(rai[:2*self.length]))
            
            inp_split = 'test' if h % 4 == 0 else 'train' # designate 25% of examples as test
            if inp_split == self.split:
                break # ok
        
        x = torch.tensor(rai[:-1], dtype=torch.long)
        y = torch.tensor(rai[1:], dtype=torch.long)
        
        # we only want to predict at output locations, mask out the loss at the input locations
        y[:2*self.length-1] = -1
        return x, y

In [5]:
class MultiplyLongDataset(Dataset):
    """ 
    Dataset for the Multiply problem. E.g. for problem length 3:
    12 * 333 = 345
    Input: 0 1 2 3 3 3 -> Output: 0 0 3 9 9 6
    Which will feed into the transformer concatenated as:
    input:  0 1 2 3 3 3 0 0 3 9 9
    output: I I I I I 0 0 3 9 9 6
    where I is "ignore", as the transformer is reading the input sequence
    """

    def __init__(self, split, length=3):
        assert split in {'train', 'test'}
        self.split = split
        self.length = length
    
    def __len__(self):
        return 10000 # ...
    
    def get_vocab_size(self):
        return 10
    
    def get_block_size(self):
        # the length of the sequence that will feed into transformer, 
        # containing concatenated input and the output, but -1 because
        # the transformer starts making predictions at the last input element
        return 2*self.length + self.length*2*self.length + 2*self.length - 1

    def __getitem__(self, idx):
        while True:
            rai = random_multiply_long_instance(self.length)
            h = hash(str(rai[:2*self.length]))
            
            inp_split = 'test' if h % 4 == 0 else 'train' # designate 25% of examples as test
            if inp_split == self.split:
                break # ok
        
        x = torch.tensor(rai[:-1], dtype=torch.long)
        y = torch.tensor(rai[1:], dtype=torch.long)
        
        # we only want to predict at output locations, mask out the loss at the input locations
        y[:2*self.length + self.length*2*self.length - 1] = -1
        return x, y

In [8]:
# # print an example instance of the dataset
# train_dataset = AddDataset('train')
# test_dataset = AddDataset('test')
# x, y = train_dataset[0]

# print (x)
# for a, b in zip(x,y):
#     print(int(a),int(b))

In [6]:
train_dataset_m = MultiplyDataset('train')
test_dataset_m = MultiplyDataset('test')
x_m, y_m = train_dataset_m[0]

print (x_m, y_m[-1].item())
for a_m, b_m in zip(x_m,y_m):
    print(int(a_m),int(b_m))

tensor([2, 9, 3, 4, 8, 0, 1, 4, 0, 6, 4]) 0
2 -1
9 -1
3 -1
4 -1
8 -1
0 1
1 4
4 0
0 6
6 4
4 0


In [7]:
train_dataset_ml = MultiplyLongDataset('train')
test_dataset_ml = MultiplyLongDataset('test')
x_ml, y_ml = train_dataset_ml[0]

print (x_ml, y_ml[-1].item())
for a_ml, b_ml in zip(x_ml,y_ml):
    print(int(a_ml),int(b_ml))

tensor([2, 3, 5, 6, 2, 9, 0, 0, 2, 1, 1, 5, 0, 0, 4, 7, 0, 0, 1, 4, 1, 0, 0, 0,
        1, 4, 7, 8, 1]) 5
2 -1
3 -1
5 -1
6 -1
2 -1
9 -1
0 -1
0 -1
2 -1
1 -1
1 -1
5 -1
0 -1
0 -1
4 -1
7 -1
0 -1
0 -1
1 -1
4 -1
1 -1
0 -1
0 -1
0 1
1 4
4 7
7 8
8 1
1 5


In [11]:
# create a GPT instance


# model_config = GPT.get_default_config()
# model_config.model_type = 'gpt-micro'
# model_config.model_type = 'gpt-nano'

# model_config.vocab_size = train_dataset.get_vocab_size()
# model_config.block_size = train_dataset.get_block_size()
# model = GPT(model_config)

In [12]:
model_config_m = GPT.get_default_config()
model_config_m.model_type = 'gpt-micro'
# model_config_m.model_type = 'gpt-nano'

model_config_m.vocab_size = train_dataset_m.get_vocab_size()
model_config_m.block_size = train_dataset_m.get_block_size()
model_m = GPT(model_config_m)

number of parameters: 0.80M


In [13]:
model_config_ml = GPT.get_default_config()
model_config_ml.model_type = 'gpt-micro'
# model_config_m.model_type = 'gpt-nano'

model_config_ml.vocab_size = train_dataset_ml.get_vocab_size()
model_config_ml.block_size = train_dataset_ml.get_block_size()
model_ml = GPT(model_config_ml)

number of parameters: 0.80M


In [14]:
# print (model_config.n_head, model_config.n_layer, model_config.n_embd)

In [15]:
# create a Trainer object

# train_config = Trainer.get_default_config()
# train_config.learning_rate = 5e-4 # the model we're using is so small that we can go a bit faster
# train_config.max_iters = 5000
# train_config.num_workers = 0
# trainer = Trainer(train_config, model, train_dataset)

In [16]:
train_config_m = Trainer.get_default_config()
train_config_m.learning_rate = 5e-3 # the model we're using is so small that we can go a bit faster
train_config_m.max_iters = 10000
train_config_m.num_workers = 0

# train_config_m.learning_rate = 1e-3  # Lower learning rate for stability
# train_config_m.max_iters = 30000     # Reduce if convergence is fast
# train_config_m.num_workers = 4       # Enable parallel data loading
# train_config_m.batch_size = 64       # Example batch size
# train_config_m.lr_scheduler = "cosine"  # Optionally use a learning rate scheduler

trainer_m = Trainer(train_config_m, model_m, train_dataset_m)

running on device cuda


In [17]:
train_config_ml = Trainer.get_default_config()
train_config_ml.learning_rate = 5e-3 # the model we're using is so small that we can go a bit faster
train_config_ml.max_iters = 10000
train_config_ml.num_workers = 0

# train_config_m.learning_rate = 1e-3  # Lower learning rate for stability
# train_config_m.max_iters = 30000     # Reduce if convergence is fast
# train_config_m.num_workers = 4       # Enable parallel data loading
# train_config_m.batch_size = 64       # Example batch size
# train_config_m.lr_scheduler = "cosine"  # Optionally use a learning rate scheduler

trainer_ml = Trainer(train_config_ml, model_ml, train_dataset_ml)

running on device cuda


In [8]:
def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}")
# trainer.set_callback('on_batch_end', batch_end_callback)


In [19]:
# trainer.run() # addition

In [20]:
trainer_m.set_callback('on_batch_end', batch_end_callback)
trainer_m.run() # multiplication

iter_dt 0.00ms; iter 0: train loss 2.33108
iter_dt 12.69ms; iter 100: train loss 2.15056
iter_dt 11.96ms; iter 200: train loss 1.94409
iter_dt 11.96ms; iter 300: train loss 1.85359
iter_dt 12.02ms; iter 400: train loss 1.91285
iter_dt 12.05ms; iter 500: train loss 1.82069
iter_dt 11.68ms; iter 600: train loss 1.83571
iter_dt 12.11ms; iter 700: train loss 1.83202
iter_dt 12.00ms; iter 800: train loss 1.77521
iter_dt 11.57ms; iter 900: train loss 1.79282
iter_dt 11.97ms; iter 1000: train loss 1.78317
iter_dt 10.18ms; iter 1100: train loss 1.73567
iter_dt 11.28ms; iter 1200: train loss 1.86182
iter_dt 12.03ms; iter 1300: train loss 1.77553
iter_dt 12.01ms; iter 1400: train loss 1.80835
iter_dt 12.00ms; iter 1500: train loss 1.82002
iter_dt 12.07ms; iter 1600: train loss 1.73743
iter_dt 12.00ms; iter 1700: train loss 1.75644
iter_dt 11.97ms; iter 1800: train loss 1.75930
iter_dt 11.99ms; iter 1900: train loss 1.72904
iter_dt 11.94ms; iter 2000: train loss 1.67961
iter_dt 12.03ms; iter 2100

In [21]:
trainer_ml.set_callback('on_batch_end', batch_end_callback)
trainer_ml.run() # multiplication long

iter_dt 0.00ms; iter 0: train loss 2.30539
iter_dt 11.44ms; iter 100: train loss 1.93719
iter_dt 11.02ms; iter 200: train loss 1.85809
iter_dt 11.67ms; iter 300: train loss 1.72666
iter_dt 12.58ms; iter 400: train loss 1.63121
iter_dt 11.22ms; iter 500: train loss 1.57343
iter_dt 13.34ms; iter 600: train loss 1.52472
iter_dt 12.73ms; iter 700: train loss 1.50868
iter_dt 12.93ms; iter 800: train loss 1.52555
iter_dt 12.71ms; iter 900: train loss 1.45786
iter_dt 12.66ms; iter 1000: train loss 1.45691
iter_dt 12.57ms; iter 1100: train loss 1.38111
iter_dt 10.63ms; iter 1200: train loss 1.44089
iter_dt 12.60ms; iter 1300: train loss 1.38610
iter_dt 12.43ms; iter 1400: train loss 1.42287
iter_dt 12.73ms; iter 1500: train loss 1.44662
iter_dt 10.76ms; iter 1600: train loss 1.39539
iter_dt 10.56ms; iter 1700: train loss 1.38269
iter_dt 11.49ms; iter 1800: train loss 1.36484
iter_dt 10.72ms; iter 1900: train loss 1.37878
iter_dt 12.62ms; iter 2000: train loss 1.37539
iter_dt 10.48ms; iter 2100

In [22]:
# now let's perform some evaluation
# model.eval()
model_m.eval()
# model_ml.eval()
None

In [10]:
def eval_add_split(trainer, split, max_batches, model, train_dataset, test_dataset):
    dataset = {'train':train_dataset, 'test':test_dataset}[split]
    n = train_dataset.length # naugy direct access shrug
    results = []
    mistakes_printed_already = 0
    loader = DataLoader(dataset, batch_size=100, num_workers=0, drop_last=False)
    #loader = DataLoader(dataset, batch_size=1, num_workers=0, drop_last=False)
    for b, (x, y) in enumerate(loader):
        x = x.to(trainer.device)
        y = y.to(trainer.device)

        inp = x[:, :2*n]
        sol = y[:, -n-1:]
        
        cat = model.generate(inp, n+1, do_sample=False) # using greedy argmax, not sampling
        sol_candidate = cat[:, -n-1:]         
        correct = (sol == sol_candidate).all(1).cpu() 
        for i in range(x.size(0)):
            results.append(int(correct[i]))
    
    rt = torch.tensor(results, dtype=torch.float)
    print("%s final score: %d/%d = %.2f%% correct" % (split, rt.sum(), len(results), 100*rt.mean()))
    return rt.sum()

# run a lot of examples from both train and test through the model and verify the output correctness


In [24]:
# with torch.no_grad():
#     train_score = eval_add_split(trainer, 'train', max_batches=50, model=model, train_dataset=train_dataset, test_dataset=test_dataset)
#     test_score  = eval_add_split(trainer, 'test',  max_batches=50, model=model, train_dataset=train_dataset, test_dataset=test_dataset)

In [11]:
with torch.no_grad():
    train_score = eval_add_split(trainer_m, 'train', max_batches=50, model=model_m, train_dataset=train_dataset_m, test_dataset=test_dataset_m)
    test_score  = eval_add_split(trainer_m, 'test',  max_batches=50, model=model_m, train_dataset=train_dataset_m, test_dataset=test_dataset_m)

NameError: name 'trainer_m' is not defined

In [26]:
with torch.no_grad():
    train_score = eval_add_split(trainer_ml, 'train', max_batches=50, model=model_ml, train_dataset=train_dataset_ml, test_dataset=test_dataset_ml)
    test_score  = eval_add_split(trainer_ml, 'test',  max_batches=50, model=model_ml, train_dataset=train_dataset_ml, test_dataset=test_dataset_ml)

train final score: 1/10000 = 0.01% correct
test final score: 3/10000 = 0.03% correct


In [12]:
model_config_m2 = GPT.get_default_config()
# model_config_m2.model_type = 'gpt-mini'
model_config_m2.model_type = 'gpt-micro'
# model_config_m2.model_type = "gpt2"
# model_config_m.model_type = 'gpt-nano'

model_config_m2.vocab_size = train_dataset_m.get_vocab_size()
model_config_m2.block_size = train_dataset_m.get_block_size()
model_m2 = GPT(model_config_m2)


train_config_m2 = Trainer.get_default_config()
train_config_m2.num_workers = 0

train_config_m2.learning_rate = 1e-3  # Lower learning rate for stability
train_config_m2.max_iters = 10000     # Reduce if convergence is fast
train_config_m2.batch_size = 4096       # Example batch size
train_config_m2.weight_decay = 0.10

trainer_m2 = Trainer(train_config_m2, model_m2, train_dataset_m)

number of parameters: 0.80M
running on device cuda


In [13]:
trainer_m2.set_callback('on_batch_end', batch_end_callback)
trainer_m2.run() # multiplication m2

iter_dt 0.00ms; iter 0: train loss 2.32143
iter_dt 156.88ms; iter 100: train loss 1.62877
iter_dt 144.56ms; iter 200: train loss 1.49800
iter_dt 145.13ms; iter 300: train loss 1.41852
iter_dt 145.67ms; iter 400: train loss 1.40050
iter_dt 145.69ms; iter 500: train loss 1.34754
iter_dt 145.95ms; iter 600: train loss 1.25344
iter_dt 145.56ms; iter 700: train loss 1.19493
iter_dt 145.99ms; iter 800: train loss 1.15087
iter_dt 146.31ms; iter 900: train loss 1.12040
iter_dt 151.63ms; iter 1000: train loss 1.07107
iter_dt 233.32ms; iter 1100: train loss 1.03723
iter_dt 146.37ms; iter 1200: train loss 1.01270
iter_dt 235.12ms; iter 1300: train loss 0.96872
iter_dt 146.32ms; iter 1400: train loss 0.92438
iter_dt 145.90ms; iter 1500: train loss 0.90211
iter_dt 146.36ms; iter 1600: train loss 0.87538
iter_dt 147.02ms; iter 1700: train loss 0.84661
iter_dt 146.31ms; iter 1800: train loss 0.82808
iter_dt 146.03ms; iter 1900: train loss 0.81627
iter_dt 146.17ms; iter 2000: train loss 0.80975
iter_d

In [14]:
model_m2.eval()
with torch.no_grad():
    train_score_m2 = eval_add_split(trainer_m2, 'train', max_batches=50, model=model_m2, train_dataset=train_dataset_m, test_dataset=test_dataset_m)
    test_score_m2  = eval_add_split(trainer_m2, 'test',  max_batches=50, model=model_m2, train_dataset=train_dataset_m, test_dataset=test_dataset_m)

train final score: 2394/1000000 = 0.24% correct
test final score: 2589/1000000 = 0.26% correct


In [96]:
from itertools import product

# Define the parameter grid
param_grid = {
    'learning_rate': [1e-3, 4.5e-3, 1e-2],
    'batch_size': [16, 32, 128, 256, 512, 1024],
    'weight_decay': [0.01, 0.1, 0.2]
}

# Get all combinations of parameters
param_combinations = list(product(param_grid['learning_rate'], param_grid['batch_size'], param_grid['weight_decay']))

best_accuracy = 0.0
best_params = None

# Loop through all combinations of parameters
for lr, batch_size, weight_decay in param_combinations:
    print(f"Testing parameters: lr={lr}, batch_size={batch_size}, weight_decay={weight_decay}")

    # Configure model and trainer
    train_config = Trainer.get_default_config()
    train_config.learning_rate = lr
    train_config.batch_size = batch_size
    train_config.weight_decay = weight_decay
    train_config.num_workers = 0  # Ensure compatibility with the system
    train_config.max_iters = 10000
    trainer = Trainer(train_config, model_m2, train_dataset_m)

    # Train the model
    trainer.run()  # You can also limit iterations here for faster search
    
    # Evaluate the model
    accuracy = eval_add_split(trainer, 'test', max_batches=100, model=model_m2, 
                              train_dataset=train_dataset_m, test_dataset=test_dataset_m).mean()

    print(f"Accuracy: {accuracy:.2f}%")

    # Update best parameters if this is the highest accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = {'learning_rate': lr, 'batch_size': batch_size, 'weight_decay': weight_decay}

# Output the best parameters and accuracy
print(f"Best parameters: {best_params}")
print(f"Best accuracy: {best_accuracy:.2f}%")


Testing parameters: lr=0.001, batch_size=16, weight_decay=0.01
running on device cuda
test final score: 22/10000 = 0.22% correct
Accuracy: 22.00%
Testing parameters: lr=0.001, batch_size=16, weight_decay=0.1
running on device cuda
test final score: 13/10000 = 0.13% correct
Accuracy: 13.00%
Testing parameters: lr=0.001, batch_size=16, weight_decay=0.2
running on device cuda
test final score: 19/10000 = 0.19% correct
Accuracy: 19.00%
Testing parameters: lr=0.001, batch_size=32, weight_decay=0.01
running on device cuda
test final score: 20/10000 = 0.20% correct
Accuracy: 20.00%
Testing parameters: lr=0.001, batch_size=32, weight_decay=0.1
running on device cuda
test final score: 26/10000 = 0.26% correct
Accuracy: 26.00%
Testing parameters: lr=0.001, batch_size=32, weight_decay=0.2
running on device cuda
test final score: 17/10000 = 0.17% correct
Accuracy: 17.00%
Testing parameters: lr=0.001, batch_size=128, weight_decay=0.01
running on device cuda
test final score: 20/10000 = 0.20% correc

KeyboardInterrupt: 

In [97]:
len(train_dataset_m)

10000